In [1]:
import tensorflow as tf
from tf_utils import * 
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Samples 1000 random mnist batches and compares which have the same label
# then search for the same amount of wrong labels
def mnist_generate_data(batch_size=20000):

    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_rxs, batch_rys = mnist.train.next_batch(batch_size)

    import numpy as np
    new_ys = np.argmax(batch_ys,1) == np.argmax(batch_rys,1)

    true_ids = np.where(new_ys == True)[0]
    num_ids = true_ids.shape[0]

    rxs = batch_rxs[true_ids]
    xs = batch_xs[true_ids]
    ys = new_ys[true_ids]

    false_ids = np.where(new_ys == False)[0]
    false_ids = false_ids[:num_ids] #prune

    rxs = np.append(rxs, batch_rxs[false_ids], axis=0)
    xs = np.append(xs, batch_xs[false_ids], axis=0)
    ys = np.append(ys, new_ys[false_ids])

    ys = dense_to_one_hot(ys) # New vector
    
    return rxs, xs, ys

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Siamese Neural Network

# %% Setup input to the network and true output label.  These are
# simply placeholders which we'll fill in later.
x = tf.placeholder(tf.float32, [None, 784])
rx = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 2])

# %% We add a new type of placeholder to denote when we are training.
# This will be used to change the way we compute the network during
# training/testing.
is_training = tf.placeholder(tf.bool, name='is_training')

# %% We'll convert our MNIST vector data to a 4-D tensor:
# N x W x H x C
x_tensor = tf.reshape(x, [-1, 28, 28, 1])
tf.image_summary('x', x_tensor)
rx_tensor = tf.reshape(rx, [-1, 28, 28, 1])
tf.image_summary('rx', rx_tensor)

# %% MNIST Convolutional Neural Net with same weights for pretraining


# %% Siamese Network
with tf.variable_scope('siamese_network') as scope:
#with tf.variable_scope("unbound_siamese_network_1") as scope:
    

    h_1 = lrelu(batch_norm(conv2d(x_tensor, 32, name='conv1'),
                           is_training, scope='bn1'), name='lrelu1')
    h_2 = lrelu(batch_norm(conv2d(h_1, 64, name='conv2'),
                           is_training, scope='bn2'), name='lrelu2')
    h_3 = lrelu(batch_norm(conv2d(h_2, 64, name='conv3'),
                           is_training, scope='bn3'), name='lrelu3')
    h_3_flat = tf.reshape(h_3, [-1, 64 * 4 * 4])

#with tf.variable_scope("unbound_siamese_network_2") as scope:
    
    scope.reuse_variables()
    
#     w1 = tf.get_variable('conv1/w')
#     b1 = tf.get_variable('conv1/b')
#     w2 = tf.get_variable('conv2/w')
#     b2 = tf.get_variable('conv2/b')
#     w3 = tf.get_variable('conv3/w')
#     b3 = tf.get_variable('conv3/b')
    
#     tf.histogram_summary('w1',w1)
#     tf.histogram_summary('b1',b1)
#     tf.histogram_summary('w2',w2)
#     tf.histogram_summary('b2',b2)
#     tf.histogram_summary('w3',w3)
#     tf.histogram_summary('b3',b3)
        
    rh_1 = lrelu(batch_norm(conv2d(rx_tensor, 32, name='conv1'),
                           is_training, scope='bn1'), name='lrelu1')
    rh_2 = lrelu(batch_norm(conv2d(rh_1, 64, name='conv2'),
                           is_training, scope='bn2'), name='lrelu2')
    rh_3 = lrelu(batch_norm(conv2d(rh_2, 64, name='conv3'),
                           is_training, scope='bn3'), name='lrelu3')
    rh_3_flat = tf.reshape(rh_3, [-1, 64 * 4 * 4])
    

d_3_flat = tf.abs(h_3_flat - rh_3_flat) # L2
h_4 = linear(d_3_flat, 2) # Weight the different parameters
y_pred = tf.nn.softmax(h_4)


# %% 

# %% Define loss/eval/training functions
cross_entropy = -tf.reduce_sum(y * tf.log(y_pred))

tf.scalar_summary('loss', cross_entropy)

train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
tf.scalar_summary('accuracy', accuracy)

# %% We now create a new session to actually perform the initialization the
# variables:
sess = tf.Session()
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter("./tmp/mnist_logs", sess.graph_def)
sess.run(tf.initialize_all_variables())

# %% We'll train in minibatches and report accuracy:
n_epochs = 100000
for epoch_i in range(n_epochs):
            
    rxs, xs, ys = mnist_generate_data()
    result = sess.run([merged, train_step], feed_dict={x: xs, rx: rxs, y: ys, is_training: True})
    
    writer.add_summary(result[0], epoch_i)
    
    if epoch_i % 50 == 0:
        rxs, xs, ys = mnist_generate_data()
        print(sess.run(accuracy,
                       feed_dict={
                           x: xs,
                           rx: rxs, 
                           y: ys,
                           is_training: False
                       }))

0.5
0.899024
0.967095
0.978489
0.983308
0.98433
0.987332


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


x,x1,y = mnist_generate_data()
plt.imshow(x[1].reshape((28,28)), cmap='gray')

In [ ]:
plt.imshow(x1[1].reshape((28,28)), cmap='gray')

In [ ]:
mnist.train.next_batch(50)

In [ ]:
h_1.name

In [ ]:
for i in tf.all_variables():
    print i.name

In [ ]:
with tf.variable_scope('test3') as scope:
    w = tf.get_variable('w',[1])
    scope.reuse_variables()
    w2 = tf.get_variable('w',[1])